In [1]:
!pip install datasets seacrowd bitsandbytes
from transformers import AutoTokenizer, Phi3ForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━

In [2]:
!pip install -U bitsandbytes
# Load pre-trained model and tokenizer
checkpoint = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = Phi3ForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency
    trust_remote_code=True,
    device_map="auto",  # Automatically map the model to available GPUs
    load_in_8bit=True,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [3]:
!pip install -U peft bitsandbytes
from peft import LoraConfig, get_peft_model, TaskType
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["qkv_proj", "o_proj"],  # Adjust based on your model architecture
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    # bnb_8bit=True  # Enable bitsandbytes 8-bit quantization
)
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [13]:
# 1. Memuat Dataset
dataset = load_dataset("SEACrowd/liputan6")
print("Dataset loaded:", dataset)
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
print("Train dataset:", train_dataset)
val_dataset = dataset["validation"].shuffle(seed=42).select(range(200))
print("Validation dataset:", val_dataset)
test_dataset = dataset["test"]
print("Test dataset:", test_dataset)

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 193883
    })
    test: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 10972
    })
    validation: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 10972
    })
})
Train dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 1000
})
Validation dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 200
})
Test dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 10972
})


In [14]:
from datasets import load_dataset
!pip install datasets
# Preprocess the dataset for summarization tasks
def preprocess_function(examples):
    inputs = examples["document"]
    targets = examples["summary"]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    # Tokenize the labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length",
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
train_tokenized_datasets = train_dataset.map(preprocess_function, batched=True)
val_tokenized_datasets = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
!pip install trl
from peft import (
        get_peft_model,
        prepare_model_for_kbit_training,
        LoraConfig
    )
from trl import SFTTrainer
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [15]:
def formatting_func(prompt):
  output = []

  for d, s in zip(prompt["src"], prompt["tgt"]):
    op = generate_prompt(d, s)
    output.append(op)

  return output


In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./phi3-liputan6-summarizer-lora",  # Directory for saving model checkpoints
    evaluation_strategy="epoch",
    learning_rate=5e-4,  # Higher learning rate for LoRA fine-tuning
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,  # Limit number of saved checkpoints
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,  # Enable mixed precision for faster training
    push_to_hub=False,
)

# Define the Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    peft_config=lora_config,
    formatting_func=formatting_func,
    # max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

# Train the model with LoRA
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-16-2c98b2280b0d>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:359: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnin

Epoch,Training Loss,Validation Loss
1,2.052500,1.890009
2,1.840200,1.857540
3,1.655600,1.868113


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to f

TrainOutput(global_step=750, training_loss=1.8656447347005207, metrics={'train_runtime': 5184.0741, 'train_samples_per_second': 0.579, 'train_steps_per_second': 0.145, 'total_flos': 3.4350774091776e+16, 'train_loss': 1.8656447347005207, 'epoch': 3.0})

In [29]:
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch
from peft import PeftModel

# Load the model and tokenizer
peft_model_id = "/content/phi3-liputan6-summarizer-lora/checkpoint-750"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")

def generate_prompt(artikel, val_dataset, output=None, eos_token="</s>"):
    instruction = """
ringkas artikel berita dibawah ini dalam bahasa indonesia:
document: {val_dataset[0]["document"]}
summary: {val_dataset[0]["summary"]}
Artikel """
    input = f"{artikel}\n"
    output = f"""Summary:
#answer:
{output + ' ' + eos_token if output else ''} """
    prompt = (" ").join([instruction, input, output])
    return prompt

# BLEU score calculation
def calculate_bleu(reference, hypothesis):
    chencherry = SmoothingFunction()
    reference = [reference.split()]
    hypothesis = hypothesis.split()
    bleu_score = sentence_bleu(reference, hypothesis, smoothing_function=chencherry.method2)
    return bleu_score

# Prediction and evaluation loop
bleu_scores = []
predictions = []
for i in range(0, 11):  # From test_dataset[0] to test_dataset[10]
    document = test_dataset[i]["document"]
    reference_summary = test_dataset[i]["summary"]

    input_prompt = generate_prompt(document, val_dataset)
    input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")

    with torch.amp.autocast('cuda'):
        generation_output = peft_model.generate(
            input_ids=input_tokens,
            max_new_tokens=100,
            do_sample=True,
            top_k=10,
            top_p=0.9,
            temperature=0.3,
            repetition_penalty=1.15,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode prediction
    output_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    predictions.append(output_text)

    # Calculate BLEU score
    bleu_score = calculate_bleu(reference_summary, output_text)
    bleu_scores.append(bleu_score)

# Display predictions and BLEU scores
for i, (prediction, bleu_score) in enumerate(zip(predictions, bleu_scores)):
    print(f"Prediction {i+1}: {prediction}")
    print(f"BLEU Score {i+1}: {bleu_score:.4f}")

# Calculate and print average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.4f}")


Prediction 1: 
ringkas artikel berita dibawah ini dalam bahasa indonesia:
'document': 'Liputan6 . com , Jombang : Maman Sugianto alias Sugik , yang masih berstatus terdakwa kasus pembunuhan Asrori , ditangguhkan penahanannya sejak Senin ( 24/11 ) malam . Penangguhan penahanannya ini dimanfaatkan Sugik untuk berkunjung ke rumah sanak saudaranya . Kedatangan Sugik disambut penuh haru bahkan sang bibi langsung jatuh pingsan . Sugik bebas bersyarat dari Lembaga Pemasyarakatan Jombang setelah menjadi korban salah tangkap dan peradilan sesat dalam kasus pembunuhan Asrori . Terdakwa lainnya yakni Devid dan Kemat , hingga kini masih dipenjara dan tengah mengajukan peninjauan kembali ( PK ) ke Mahkamah Agung . Sugik dan istrinya berharap majelis hakim lebih bijaksana untuk membebaskan dirinya dari semua tuduhan . Sebab , ia memang merasa tak membunuh Asrori seperti yang dituduhkan polisi dan jaksa [ baca : Saksi Ahli : Terdakwa Bisa Dibebaskan ] . ( UPI/Bambang Ronggo ) .'
'summary': 'Maman Sug

In [33]:
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch
from peft import PeftModel

# Load the model and tokenizer
peft_model_id = "/content/phi3-liputan6-summarizer-lora/checkpoint-750"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")

def generate_prompt(artikel, val_dataset, output=None, eos_token="</s>"):
    # Default output jika tidak diberikan
    if output is None:
        output = ""

    # Instruction menggunakan f-string
    instruction = f"""
ringkas artikel berita dibawah ini dalam bahasa indonesia:
document: {val_dataset[0]["document"]}
summary: {val_dataset[0]["summary"]}

document: {val_dataset[1]["document"]}
summary: {val_dataset[1]["summary"]}

document: {val_dataset[2]["document"]}
summary: {val_dataset[2]["summary"]}

Artikel yang akan diringkas:
document: {artikel}
"""
    # Output format
    output = f"""Summary:
#answer:
{output + ' ' + eos_token if output else ''}"""

    # Gabungkan instruction dan output
    prompt = (" ").join([instruction, output])
    return prompt



# BLEU score calculation
def calculate_bleu(reference, hypothesis):
    chencherry = SmoothingFunction()
    reference = [reference.split()]
    hypothesis = hypothesis.split()
    bleu_score = sentence_bleu(reference, hypothesis, smoothing_function=chencherry.method2)
    return bleu_score

# Prediction and evaluation loop
bleu_scores = []
predictions = []
for i in range(0, 11):  # From test_dataset[0] to test_dataset[10]
    document = test_dataset[i]["document"]
    reference_summary = test_dataset[i]["summary"]

    input_prompt = generate_prompt(document, val_dataset)
    input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")

    with torch.amp.autocast('cuda'):
        generation_output = peft_model.generate(
            input_ids=input_tokens,
            max_new_tokens=100,
            do_sample=True,
            top_k=10,
            top_p=0.9,
            temperature=0.3,
            repetition_penalty=1.15,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode prediction
    output_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    predictions.append(output_text)

    # Calculate BLEU score
    bleu_score = calculate_bleu(reference_summary, output_text)
    bleu_scores.append(bleu_score)

# Display predictions and BLEU scores
for i, (prediction, bleu_score) in enumerate(zip(predictions, bleu_scores)):
    print(f"Prediction {i+1}: {prediction}")
    print(f"BLEU Score {i+1}: {bleu_score:.4f}")

# Calculate and print average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.4f}")


Prediction 1: 
ringkas artikel berita dibawah ini dalam bahasa indonesia:
document: Liputan6 . com , Jakarta : Mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita mengaku telah menerima panggilan Kejaksaan Agung . Tapi , dia keberatan diperiksa sebagai tersangka dalam Kasus Technical Assistance Contract ( TAC ) antara Pertamina dan PT Ustraindo Petro Gas . Dia menuding status yang dikeluarkan Kejagung lebih bermuatan politis dibanding pertimbangan hukum . Pernyataan itu diungkapkan Ginandjar dalam jumpa pers yang digelar di kediamannya di Jalan Widya Chandra V Nomor 26 , Jakarta Selatan , Selasa ( 20/3 ) . Petinggi Partai Golongan Karya itu mengaku tak pernah ingkar janji soal pemanggilan Kejagung . Tapi , dia keberatan dengan status tersangka dalam kasus korupsi yang merugikan negara sebesar US$ 17 , 4 juta itu . Sebab , dia hanya melaksanakan kebijakan pemerintah selama proses persiapan dan perundingan . Dia menolak dituding berperan aktif dalam TAC tersebut . Ginandjar men

In [34]:
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch
from peft import PeftModel

# Load the model and tokenizer
peft_model_id = "/content/phi3-liputan6-summarizer-lora/checkpoint-750"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")

def generate_prompt(artikel, val_dataset, output=None, eos_token="</s>"):
    # Default output jika tidak diberikan
    if output is None:
        output = ""

    # Instruction menggunakan f-string
    instruction = f"""
ringkas artikel berita dibawah ini dalam bahasa indonesia:
document: {val_dataset[0]["document"]}
summary: {val_dataset[0]["summary"]}

Artikel yang akan diringkas:
document: {artikel}
"""
    # Output format
    output = f"""Summary:
#answer:
{output + ' ' + eos_token if output else ''}"""

    # Gabungkan instruction dan output
    prompt = (" ").join([instruction, output])
    return prompt



# BLEU score calculation
def calculate_bleu(reference, hypothesis):
    chencherry = SmoothingFunction()
    reference = [reference.split()]
    hypothesis = hypothesis.split()
    bleu_score = sentence_bleu(reference, hypothesis, smoothing_function=chencherry.method2)
    return bleu_score

# Prediction and evaluation loop
bleu_scores = []
predictions = []
for i in range(0, 11):  # From test_dataset[0] to test_dataset[10]
    document = test_dataset[i]["document"]
    reference_summary = test_dataset[i]["summary"]

    input_prompt = generate_prompt(document, val_dataset)
    input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")

    with torch.amp.autocast('cuda'):
        generation_output = peft_model.generate(
            input_ids=input_tokens,
            max_new_tokens=100,
            do_sample=True,
            top_k=10,
            top_p=0.9,
            temperature=0.3,
            repetition_penalty=1.15,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode prediction
    output_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    predictions.append(output_text)

    # Calculate BLEU score
    bleu_score = calculate_bleu(reference_summary, output_text)
    bleu_scores.append(bleu_score)

# Display predictions and BLEU scores
for i, (prediction, bleu_score) in enumerate(zip(predictions, bleu_scores)):
    print(f"Prediction {i+1}: {prediction}")
    print(f"BLEU Score {i+1}: {bleu_score:.4f}")

# Calculate and print average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.4f}")


Prediction 1: 
ringkas artikel berita dibawah ini dalam bahasa indonesia:
document: Liputan6 . com , Jakarta : Mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita mengaku telah menerima panggilan Kejaksaan Agung . Tapi , dia keberatan diperiksa sebagai tersangka dalam Kasus Technical Assistance Contract ( TAC ) antara Pertamina dan PT Ustraindo Petro Gas . Dia menuding status yang dikeluarkan Kejagung lebih bermuatan politis dibanding pertimbangan hukum . Pernyataan itu diungkapkan Ginandjar dalam jumpa pers yang digelar di kediamannya di Jalan Widya Chandra V Nomor 26 , Jakarta Selatan , Selasa ( 20/3 ) . Petinggi Partai Golongan Karya itu mengaku tak pernah ingkar janji soal pemanggilan Kejagung . Tapi , dia keberatan dengan status tersangka dalam kasus korupsi yang merugikan negara sebesar US$ 17 , 4 juta itu . Sebab , dia hanya melaksanakan kebijakan pemerintah selama proses persiapan dan perundingan . Dia menolak dituding berperan aktif dalam TAC tersebut . Ginandjar men